In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from skimage.transform import estimate_transform

Setting environment for Gordon


In [33]:
stack = 'MD585'

In [34]:
if os.path.exists('/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/'+stack):
    os.system('rm -r ' + '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/'+stack)

In [35]:
transforms = pickle.load(open(os.environ['DATA_DIR'] + '/%s_finalTransfParams.pkl' % stack, 'r'))

In [36]:
new_annotation_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/'

In [37]:
def transfer_latest_to_original(stack, sec, username):
    
    res = DataManager.load_annotation(stack=stack, section=sec, username=username, 
                                                    annotation_rootdir='/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped/')
    if res is None:
        return
    
    annotations, _, timestamp = res
    
    cropbox_xmin, cropbox_ymin = brainstem_bbox_lookup[stack][:2]
    
    new_annotations = []
    for pol in annotations:
        new_pol = pol.copy()
        if len(pol['vertices']) == 0:
            continue        
        
        vs = np.dot(transforms[sec], 
               (np.dot(np.diag([1./32, 1./32, 1]), 
                       np.c_[pol['vertices'], np.ones((len(pol['vertices']),))].T) +\
                        np.array([cropbox_xmin, cropbox_ymin, 0])[:,np.newaxis]))[:2].T
        vs *= 32
        new_pol['vertices'] = vs
        
        vs = np.dot(transforms[sec],  
               (np.dot(np.diag([1./32, 1./32, 1]), 
                       np.r_[new_pol['labelPos'],1]) + [cropbox_xmin, cropbox_ymin, 0]))[:2]
        vs *= 32
        new_pol['labelPos'] = np.squeeze(vs)
        
        new_annotations.append(new_pol)
        
    DataManager.save_annotation(new_annotations, stack, sec, username=username, timestamp=timestamp, 
                                annotation_rootdir=new_annotation_rootdir)

In [38]:
# transfer current labelings to labelings on originalLossless

for sec in range(section_number_lookup[stack]):
    transfer_latest_to_original(stack, sec, username='yuncong')
    transfer_latest_to_original(stack, sec, username='autoAnnotate')